<p><center> <a href="../Start_Here.ipynb"> Home Page</a> </center> </p> 
<div>
    <span style="float: left; width:20%; text-align: left;"><a href="06-DDP_Mixed_Precision.ipynb" >Previous Notebook </a></span>
    <span style="float: left; width:75%; text-align: right;"><a href="04-ZeRO.ipynb" >Next Notebook </a></span>
    
</div>

# Memory Format

---

## Channels Last Memory Format in PyTorch

Channels last memory format is an alternative way of ordering NCHW tensors in memory preserving dimensions ordering. Channels last tensors ordered in such a way that channels become the densest dimension (aka storing images pixel-per-pixel).

For example, classic (contiguous) storage of NCHW tensor (in our case it is two 4x4 images with 3 color channels) look like this:

![classic_memory_format](images/classic_memory_format.png)

Channels last memory format orders data differently:

![channels_last_memory_format](images/channels_last_memory_format.png)


Pytorch supports memory formats (and provides back compatibility with existing models including eager, JIT, and TorchScript) by utilizing  existing strides structure.
For example, `10x3x16x16` batch in Channels last format will have strides equal to `(768, 1, 48, 3)`.

<div class="alert alert-info"><h4>Note</h4><p>NCHW stands for: batch N, channels C, depth D, height H, width W. It is a way to store multidimensional arrays / data frames / matrix into memory, which can be considered as a 1-D array.</p></div>





### Classic PyTorch contiguous tensor

In [ ]:
import torch

N, C, H, W = 10, 3, 32, 32
x = torch.empty(N, C, H, W)
print(x.shape)  # Outputs: (10, 3, 32, 32) as dimensions order preserved
x.stride()  # Ouputs: (3072, 1024, 32, 1)

**Conversion operator**

In [ ]:
x = x.to(memory_format=torch.channels_last)
print(x.shape)  # Outputs: (10, 3, 32, 32) as dimensions order preserved
x.stride()  # Outputs: (3072, 1, 96, 3)

**Back to contiguous**

In [ ]:
x = x.to(memory_format=torch.contiguous_format)
x.stride()  # Outputs: (3072, 1024, 32, 1)

## Performance Gain 

The most significant performance gains are observed on NVIDIA’s hardware with Tensor Cores support running on reduced precision (`torch.float16`). We are able to archive over 22% perf gains with channels last comparing to contiguous format, both while utilizing AMP (Automated Mixed Precision) training scripts.

### Launch command

**You need ImageNet to execute the code.**

In [ ]:
!python ../source_code/main_amp.py -a resnet50 --b 200 --workers 16 --opt-level O2 --channels-last true ../source_code/data

---
## References

- Vitaly Fedyunin <https://github.com/VitalyFedyunin>
- https://github.com/apache/incubator-mxnet/issues/5778
- https://oneapi-src.github.io/oneDNN/dev_guide_understanding_memory_formats.html

<div>
    <span style="float: left; width:20%; text-align: left;"><a href="06-DDP_Mixed_Precision.ipynb" >Previous Notebook </a></span>
    <span style="float: left; width:75%; text-align: right;"><a href="04-ZeRO.ipynb" >Next Notebook </a></span>
</div>
<br/>
<p><center> <a href="../Start_Here.ipynb"> Home Page</a> </center> </p> 